# Notebook 05 - AEP Network

Sam Welch  
November 25, 2025

In [ ]:
library(STOPeData)
library(DT)
library(dplyr)



Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: viridisLite


Attaching package: 'tidygraph'

...

The following objects are masked from 'package:dplyr':

    between, first, last


Attaching package: 'eDataDRF'

The following objects are masked from 'package:STOPeData':

    abbreviate_string, altitude_units_vocabulary,
    analytical_protocols_vocabulary, areas_vocabulary,
    coordinate_systems_vocabulary, countries_vocabulary,
    dummy_parameters_vocabulary, environ_compartments_sub_vocabulary,
    environ_compartments_vocabulary, extraction_protocols_vocabulary,
    fractionation_protocols_vocabulary, gender_vocabulary,
    generate_protocol_id, geographic_features_sub_vocabulary,
    geographic_features_vocabulary, get_dataset_display_name,
    initialise_biota_tibble, initialise_campaign_tibble,
    initialise_compartments_tibble, initialise_CREED_data_tibble,
    initialise_CREED_scores_tibble, initialise_measurements_tibble,
    initialise_methods_tibble, initialise_parameters_tibble,
    initialise_references_tibble, initialise_samples_tibble,
    initialise_sites_tibble, lifestage_vocabulary,
    measured_categories_vocabulary, measured_flags_vocabulary,
    measured_types_vocabular

here() starts at C:/Users/SAW/Local Documents/EXPECT AEP/EXPECT_AEP_R_Project

ℹ Loading EXPECT_AEP_R_Project

In [ ]:
# load in any data we need from the targets workflow
literature_merged_data <- tar_read(load_literature_pqt)
literature_reference_data <- tar_read(reference_data)
literature_campaign_data <- tar_read(campaign_data)
literature_sites_data <- tar_read(sites_data)
literature_qc <- tar_read(data_quality_report)
wgs84_geo <- tar_read(wgs84_geography)

species_names <- eDataDRF::species_names_vocabulary()
species_lookup <- species_names |>
  with(setNames(SPECIES_COMMON_NAME, SPECIES_NAME))


In [ ]:

copper_toxicity_thresholds <- tar_read(copper_toxicity_thresholds)
# Define threshold colors based on the report ----
threshold_colours <- c(
  "Background (I)" = "#0070C0", # Blue
  "Good (II)" = "#00B050", # Green
  "Moderate (III)" = "#FFC000", # Yellow
  "Poor (IV)" = "#FF6600", # Orange
  "Very Poor (V)" = "#FF0000" # Red
)

compartment_colours <- c(
  # Aquatic ----
  "Freshwater" = "#4A90E2", # clear blue
  "Marine/Salt Water" = "#1E5A8E", # deep ocean blue
  "Brackish/Transitional Water" = "#6BA5C8", # mid-blue (between fresh/marine)
  "Groundwater" = "#2C5F7B", # darker, subdued blue
  "Wastewater" = "#8B7355", # murky brown
  "Liquid Growth Medium" = "#9FD8CB", # pale greenish-blue
  "Rainwater" = "#B3D9FF", # light sky blue
  "Stormwater" = "#607D8B", # grey-blue
  "Leachate" = "#6B5344", # dark muddy brown
  "Aquatic Sediment" = "#EBCF1B", # your existing yellow
  "Porewater" = "#7FA0B8", # muted blue-grey
  "Sludge" = "#67AB33", # your existing green

  # Atmospheric ----
  "Indoor Air" = "#E8F4F8", # very pale blue-grey
  "Outdoor Air" = "#87CEEB", # sky blue

  # Terrestrial ----
  "Terrestrial Biological Residue" = "#8B6F47", # organic brown
  "Soil H Horizon (Peat)" = "#3E2723", # very dark brown
  "Soil O Horizon (Organic)" = "#5D4037", # dark organic brown
  "Soil A Horizon (Topsoil)" = "#795548", # medium brown
  "Soil E Horizon (Mineral)" = "#A1887F", # light greyish-brown
  "Soil S Horizon (Mineral)" = "#BCAAA4", # pale brown-grey
  "Soil C Horizon (Parent Material)" = "#D7CCC8", # very pale brown
  "Soil R Horizon (Bedrock)" = "#9E9E9E", # grey

  # Biota ----
  "Biota, Terrestrial" = "#558B2F", # forest green
  "Biota, Aquatic" = "#00695C", # teal
  "Biota, Atmospheric" = "#B39DDB", # light purple (birds/insects)
  "Biota, Other" = "#9C27B0" # distinct purple
)

# Helper function for sample sizes
calculate_sample_sizes <- function(data) {
  data |>
    group_by(
      REFERENCE_ID,
      OCEAN_COUNTRY,
      SITE_GEOGRAPHIC_FEATURE,
      ENVIRON_COMPARTMENT_SUB
    ) |>
    summarise(total_n = sum(MEASURED_N, na.rm = TRUE), .groups = "drop")
}

# Create sub-compartment letter codes ----
subcomp_codes <- c(
  # Aquatic
  "Freshwater" = "F",
  "Marine/Salt Water" = "M",
  "Brackish/Transitional Water" = "B",
  "Groundwater" = "G",
  "Wastewater" = "WW",
  "Liquid Growth Medium" = "LGM",
  "Rainwater" = "R",
  "Stormwater" = "SW",
  "Leachate" = "L",
  "Aquatic Sediment" = "AS",
  "Porewater" = "P",
  "Sludge" = "Sl",
  # Atmospheric
  "Indoor Air" = "IA",
  "Outdoor Air" = "OA",
  # Terrestrial
  "Terrestrial Biological Residue" = "TBR",
  "Soil H Horizon (Peat)" = "H",
  "Soil O Horizon (Organic)" = "O",
  "Soil A Horizon (Topsoil)" = "A",
  "Soil E Horizon (Mineral)" = "E",
  "Soil S Horizon (Mineral)" = "S",
  "Soil C Horizon (Parent Material)" = "C",
  "Soil R Horizon (Bedrock)" = "R",
  # Biota
  "Biota, Terrestrial" = "BT",
  "Biota, Aquatic" = "BA",
  "Biota, Atmospheric" = "BAm",
  "Biota, Other" = "BO"
)


# Network Diagram

In [ ]:

unique_compartments_and_sites <- literature_merged_data |>
  select(
    SITE_GEOGRAPHIC_FEATURE,
    SITE_GEOGRAPHIC_FEATURE_SUB,
    ENVIRON_COMPARTMENT,
    ENVIRON_COMPARTMENT_SUB,
    MEASURED_N
  ) |>
  distinct() |>
  filter(if_all(everything(), ~ !is.na(.))) |>
  group_by(
    SITE_GEOGRAPHIC_FEATURE,
    SITE_GEOGRAPHIC_FEATURE_SUB,
    ENVIRON_COMPARTMENT,
    ENVIRON_COMPARTMENT_SUB
  ) |>
  reframe(sum_n = sum(MEASURED_N)) |>
  mutate(compartment_name_n = glue::glue("{ENVIRON_COMPARTMENT_SUB}\n{sum_n}"))


# TODO: This is very speculative and just generated by AI. I haven't yet reviewed it to see what's missing.
# Copper flow network edges ----
copper_flow_edges <- tribble(
  ~from                         , ~to                           , ~flow_type                , ~plausibility , ~magnitude   , ~evidence_quality , ~description                                                ,

  # Atmospheric deposition to surface waters ----
  "Atmospheric"                 , "Freshwater"                  , "deposition"              , 0.95          , "medium"     , "high"            , "Wet and dry deposition from atmospheric particulates"      ,
  "Atmospheric"                 , "Brackish/Transitional Water" , "deposition"              , 0.95          , "medium"     , "high"            , "Wet and dry deposition to coastal/estuarine waters"        ,
  "Atmospheric"                 , "Aquatic Sediment"            , "deposition"              , 0.90          , "low"        , "medium"          , "Direct deposition to exposed sediments"                    ,
  "Atmospheric"                 , "Soil O Horizon (Organic)"    , "deposition"              , 0.95          , "medium"     , "high"            , "Atmospheric deposition to terrestrial surfaces"            ,

  # Terrestrial to aquatic flows ----
  "Soil O Horizon (Organic)"    , "Freshwater"                  , "runoff"                  , 0.95          , "high"       , "high"            , "Surface runoff and leaching from soils"                    ,
  "Soil O Horizon (Organic)"    , "Groundwater"                 , "percolation"             , 0.85          , "medium"     , "medium"          , "Downward migration through soil profile"                   ,
  "Biota, Terrestrial"          , "Soil O Horizon (Organic)"    , "excretion_decomposition" , 0.90          , "low"        , "medium"          , "Excretion and decomposition of terrestrial organisms"      ,

  # Groundwater to surface water ----
  "Groundwater"                 , "Freshwater"                  , "baseflow"                , 0.90          , "medium"     , "high"            , "Groundwater discharge to streams/rivers"                   ,
  "Groundwater"                 , "Brackish/Transitional Water" , "discharge"               , 0.75          , "low"        , "medium"          , "Submarine groundwater discharge to coastal waters"         ,

  # Freshwater flows ----
  "Freshwater"                  , "Brackish/Transitional Water" , "advection"               , 0.95          , "high"       , "high"            , "River discharge to estuaries"                              ,
  "Freshwater"                  , "Aquatic Sediment"            , "sedimentation"           , 0.95          , "high"       , "high"            , "Particulate settling from water column"                    ,
  "Freshwater"                  , "Biota, Aquatic"              , "uptake"                  , 0.95          , "medium"     , "high"            , "Bioaccumulation by freshwater organisms"                   ,

  # Brackish/transitional water flows ----
  "Brackish/Transitional Water" , "Aquatic Sediment"            , "sedimentation"           , 0.95          , "high"       , "high"            , "Particulate settling in estuarine environments"            ,
  "Brackish/Transitional Water" , "Biota, Aquatic"              , "uptake"                  , 0.95          , "medium"     , "high"            , "Bioaccumulation by estuarine/coastal organisms"            ,

  # Sediment processes ----
  "Aquatic Sediment"            , "Freshwater"                  , "resuspension"            , 0.85          , "medium"     , "high"            , "Sediment resuspension and porewater release"               ,
  "Aquatic Sediment"            , "Brackish/Transitional Water" , "resuspension"            , 0.85          , "medium"     , "high"            , "Sediment resuspension in coastal/estuarine areas"          ,
  "Aquatic Sediment"            , "Biota, Aquatic"              , "uptake"                  , 0.90          , "low"        , "medium"          , "Benthic organism uptake from sediments"                    ,
  "Aquatic Sediment"            , "Groundwater"                 , "diagenesis"              , 0.70          , "low"        , "low"             , "Deep burial and diagenetic processes"                      ,

  # Biota cycling ----
  "Biota, Aquatic"              , "Aquatic Sediment"            , "excretion_decomposition" , 0.95          , "medium"     , "high"            , "Fecal pellets, mortality, decomposition"                   ,
  "Biota, Aquatic"              , "Freshwater"                  , "excretion"               , 0.90          , "low"        , "medium"          , "Dissolved excretion by aquatic organisms"                  ,
  "Biota, Aquatic"              , "Brackish/Transitional Water" , "excretion"               , 0.90          , "low"        , "medium"          , "Dissolved excretion in coastal waters"                     ,
  "Biota, Aquatic"              , "Biota, Terrestrial"          , "trophic_transfer"        , 0.80          , "low"        , "medium"          , "Predation by terrestrial organisms (e.g., birds, mammals)" ,

  # Wastewater flows ----
  "Wastewater"                  , "Freshwater"                  , "discharge"               , 0.90          , "high"       , "high"            , "Treated or untreated wastewater discharge"                 ,
  "Wastewater"                  , "Brackish/Transitional Water" , "discharge"               , 0.80          , "medium"     , "medium"          , "Coastal wastewater discharge"                              ,
  "Wastewater"                  , "Sludge"                      , "treatment"               , 0.95          , "high"       , "high"            , "Copper partitioning to sludge during treatment"            ,

  # Sludge applications ----
  "Sludge"                      , "Soil O Horizon (Organic)"    , "application"             , 0.85          , "medium"     , "medium"          , "Agricultural application of biosolids"                     ,
  "Sludge"                      , "Freshwater"                  , "discharge"               , 0.60          , "low"        , "low"             , "Illegal or emergency discharge (historical)"               ,

  # Terrestrial-atmospheric ----
  "Soil O Horizon (Organic)"    , "Atmospheric"                 , "resuspension"            , 0.70          , "low"        , "low"             , "Wind erosion and dust generation"                          ,
  "Biota, Terrestrial"          , "Atmospheric"                 , "volatilization"          , 0.40          , "negligible" , "low"             , "Unlikely for copper but included for completeness"
)

# Add reverse flow indicators for bidirectional processes if needed ----
# don't work yet
# copper_flow_edges |> left_join(unique_compartments_and_sites |> select(ENVIRON_COMPARTMENT_SUB, compartment_name_n), by = "ENVIRON_COMPARTMENT_SUB")

# Summary statistics ----
# copper_flow_edges |>
#   count(flow_type, sort = TRUE)

# copper_flow_edges |>
#   count(magnitude)

# Then plot
graph <- as_tbl_graph(copper_flow_edges)


In [ ]:

ggraph(graph, layout = "tree") +
  geom_edge_diagonal2(
    aes(label = flow_type),
    colour = "darkgrey",
    label_colour = "black",
    strength = 0.8,
    check_overlap = TRUE,
    angle_calc = "along",
    arrow = arrow(length = unit(4, 'mm')),
    start_cap = circle(10, 'mm'),
    end_cap = circle(10, 'mm')
  ) +
  geom_node_point(size = 20, aes(colour = name)) +
  geom_node_text(aes(label = str_wrap(name, 10)), size = 3, ) +
  scale_fill_manual(values = compartment_colours) +
  theme_void() +
  theme(legend.position = "none")


data's fill values.